<a href="https://colab.research.google.com/github/gustavomachin/Linear_Programming_PuLP/blob/master/Diplomatura_UDA/BA_ProgramacionLineal_Cartera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![logo](https://github.com/cristiandarioortegayubro/BA/blob/main/dba.png?raw=true)

<center>
<font color="#41a8c7">
<h2><b>
Programación Lineal - Problema de Selección de Cartera

</font>
</center>

## Instalar y cargar las librerías necesarias

In [1]:
! pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.2 MB 3.8 MB/s 


In [2]:
from pulp import *

In [3]:
import pandas as pd

In [4]:
import plotly.express as px

## Resumen del caso planteado

Se buscará determinar la combinación de activos financieros disponibles en un mercado de capitales que maximice la rentabilidad de la inversión para la empresa. 

El gerente de finanzas quiere conocer la combinación de activos financieros que maximice la rentabilidad de 100.000 dólares que debe invertir. Con base en las inversiones actuales de la empresa, el analista bursátil recomienda que todas las nuevas inversiones se hagan en la industria petrolera, la industria siderúrgica o en bonos del gobierno. Particularmente, el analista identificó cinco oportunidades de inversión y estimó sus tasas rendimiento anuales.

**Rendimientos esperados**

| Activo Financiero | Rendimiento esperado anual (%) | 
|---------------|---------|
|Petrolera 1 |7,3|
|Petrolera 2  |10,3 | 
|Metalúrgica 1  |6,3 | 
|Metalúrgica 2  |7,5 | 
|Bonos del Gobierno |4 | 

La gerencia impuso los siguientes lineamientos de inversión: 1) Ninguna industria (petrolera o siderúrgica) debe recibir más del 50% de la inversión (50,000 dólares). 2) Los bonos del gobierno deben constituir por lo menos 25% de las inversiones en la industria metalúrgica. 3) La inversión en P2, inversión de alto rendimiento pero con alto riesgo, no puede constituir más de 60% de la inversión total en la industria petrolera. 

Se intenta maximizar el rendimiento esperado sujeto a las restricciones impuestas por el presupuesto y la gerencia.


## Definir el modelo

Se contruye el modelo, que busque minimizar (LpMinimize) o maximizar (LpMaximize). En este caso es un problema de maximización de la rentabilidad de la inversión de la empresa, es decir, el rendimiento esperado.

In [5]:
modelo = LpProblem("Problema_de_Cartera", sense=LpMaximize)

##Crear las variables de decisión

Para este problema son las proporciones (participación en porcentaje) de la inversión total que se invertirá en cada activo financiero. Siendo `P1` el porcentaje invertido en la Petrolera 1, `P2` el porcentaje invertido en la Petrolera 2, `M1` el porcentaje invertido en la Metalúrgica 1,`M2` el porcentaje invertido en la Metalúrgica 2 y `BG` el porcentaje invertido en Bonos del Gobierno. 

El parámetro `lowBound=0` indica que no se pueden invertir proporciones negarivas. Por otra parte, el parámetro `cat="Continuous"` expresa que la proporción a invertir en cada activo financiero es un número con decimales, este parámetro toma valores continuos o reales por defecto.

In [6]:
P1 = LpVariable("P1", lowBound=0, cat="Continuous")
P2 = LpVariable("P2", lowBound=0, cat="Continuous")
M1 = LpVariable("M1", lowBound=0, cat="Continuous")
M2 = LpVariable("M2", lowBound=0, cat="Continuous")
BG = LpVariable("BG", lowBound=0, cat="Continuous")

##Definir la función objetivo 

El rendimiento de la cartera es el promedio ponderado por las participaciones de los rendimientos de los activos financieros que la componen.

Se agrega la función objetivo al modelo usando el operador de asignación `+=`

In [7]:
modelo += 0.073*P1 + 0.103*P2 + 0.063*M1 + 0.075*M2 + 0.04*BG 

##Agregar las restricciones

Las restricciones, así como el objetivo, agregados al modelo también deben estar expresadas en términos de las variables de decisión. Del mismo modo que definimos la función objetivo, se usa el operador de asignación `+=`, de este modo lo que se encuentra a la derecha de la expresión matemática se almacena (agrega) en la variable "modelo".

In [8]:
#inversión total
modelo += P1 + P2 + M1 + M2 + BG == 1
#inversión en industria petrolera
modelo += P1 + P2 <= 0.5
#inversión en industria metalúrgica
modelo += M1 + M2 <= 0.5 
#inversión en bonos del gobierno (BG)
modelo += BG >= 0.25*(M1+M2)
#inversión en activos de alto riesgo (P2)
modelo += P2 <= 0.6*(P1+P2)

##Resolver el modelo

Una vez definido el modelo, las variables de decisión, función objetivo y restricciones se procede a resolver el modelo usando algún método de optimización. De este modo, la mejor solución posible es identificada. 

In [9]:
#Resolver el modelo:
modelo.solve()
#Comprobar el status de la solución alcanzada:
print ("Status:", LpStatus[modelo.status])

Status: Optimal


`LpStatus` muestra el status de la solución alcanzada, en este caso, la solución alcanzada es la óptima.

In [10]:
print("Invertir en P1: ", P1.varValue * 100, "%")
print("Invertir en P2: ", P2.varValue * 100, "%")
print("Invertir en M1: ", M1.varValue * 100, "%")
print("Invertir en M2: ", M2.varValue * 100, "%")
print("Invertir en BG: ", BG.varValue * 100, "%")

Invertir en P1:  20.0 %
Invertir en P2:  30.0 %
Invertir en M1:  0.0 %
Invertir en M2:  40.0 %
Invertir en BG:  10.0 %


La solución que me permite maximizar el rendimiento esperado es invertir del capital inivial, el 20% en la Petrolera 1, el 30% en la Petrolera 2, el 40% en la Metalúrgica 2 y el 10% en Bonos del Gobierno. 

In [11]:
print("Máximo rendimiento esperado: ", value(modelo.objective) * 100, "%")

Máximo rendimiento esperado:  7.95 %


El rendimiento esperado de la cartera según las participaciones obtenidas es del 7.95%.

##Graficar

In [12]:
datos = pd.DataFrame({"P1": P1.varValue, "P2": P2.varValue, "M1": M1.varValue, "M2": M2.varValue, "BG": BG.varValue}, index=[0])
datos = datos.T
datos.columns = ["Participaciones"]
datos

,Participaciones
P1,0.2
P2,0.3
M1,0.0
M2,0.4
BG,0.1


In [13]:
px.pie(datos, 
       values="Participaciones", 
       names=datos.index, 
       title="Máximo rendimiento esperado",
       hole=.4,
       color_discrete_sequence = px.colors.sequential.Turbo)